In [1]:
import numpy as np
import torch 
import torch.optim as optim
import json
import pandas as pd
from datetime import datetime
import pdb 
from google.colab import drive
import pandas as pd
from psutil import virtual_memory
import os
from zipfile import ZipFile
!pip install wandb --upgrade
import wandb

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
     |████████████████████████████████| 180 kB 64.2 MB/s 
     |████████████████████████████████| 97 kB 7.2 MB/s 
     |████████████████████████████████| 139 kB 51.0 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=3a2a17bd464ad1e4ff403ce95dfd3a63e968e35375854d7e352a5f7450e134e0
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=d6fc42cefb33ac428d98914438d1a1c3493ad53172786d224f8019be65612719
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [2]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Data Setup

In [4]:
class education_dataset(torch.utils.data.Dataset):

  def __init__(self, data_path):
    data = pd.read_csv(data_path)
    data = data.to_numpy()

    self.X = data[:, :-1].astype(np.float32)
    self.Y = data[:, -1].astype(np.int_) 

    self.X = torch.from_numpy(self.X)
    self.Y= torch.from_numpy(self.Y)
    
  def __len__(self):
    return len(self.Y)
  
  def __getitem__(self, index):
    return(self.X[index, :], self.Y[index])

In [5]:
def create_data_loader(dataset, shuffle, batch_size = 128, num_workers = 1, pin_memory = True):
  data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                                  batch_size = batch_size,
                                                  shuffle = shuffle,
                                                  num_workers = num_workers,
                                                  pin_memory = pin_memory)
  return data_loader

#Model Setup

In [6]:
class ANNModel(torch.nn.Module):
  def __init__(self, dropout_prob, hidden_layer_size_0, hidden_layer_size_1,hidden_layer_size_2,hidden_layer_size_3,hidden_layer_size_4,hidden_layer_size_5, classes):
    super(ANNModel, self).__init__()

    self.network = torch.nn.Sequential(torch.nn.Linear(13, hidden_layer_size_0), 
                torch.nn.ReLU(), 
                torch.nn.BatchNorm1d(hidden_layer_size_0),
                torch.nn.Dropout(dropout_prob),
                torch.nn.Linear(hidden_layer_size_0, hidden_layer_size_1),
                torch.nn.ReLU(),
                torch.nn.BatchNorm1d(hidden_layer_size_1),
                torch.nn.Dropout(dropout_prob),
                torch.nn.Linear(hidden_layer_size_1, hidden_layer_size_2),
                torch.nn.ReLU(),
                torch.nn.BatchNorm1d(hidden_layer_size_2),
                torch.nn.Dropout(dropout_prob),
                torch.nn.Linear(hidden_layer_size_2, hidden_layer_size_3),
                torch.nn.ReLU(),
                torch.nn.BatchNorm1d(hidden_layer_size_3),
                torch.nn.Dropout(dropout_prob),
                torch.nn.Linear(hidden_layer_size_3, hidden_layer_size_4),
                torch.nn.ReLU(),
                torch.nn.BatchNorm1d(hidden_layer_size_4),
                torch.nn.Dropout(dropout_prob),
                torch.nn.Linear(hidden_layer_size_4, hidden_layer_size_5),
                torch.nn.ReLU(),
                torch.nn.BatchNorm1d(hidden_layer_size_5),
                torch.nn.Dropout(dropout_prob),
                torch.nn.Linear(hidden_layer_size_5, classes)
                )

  def forward(self, data):
    x = self.network(data)
    return(x)

In [7]:
def train_model(model, train_loader, optimizer, loss_function, epoch):
  model.train()
  running_loss = 0.0 

  for features, label in train_loader:
    optimizer.zero_grad()
    features, label = features.to(device), label.to(device)

    output = model(features)

    loss = loss_function(output, label)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    torch.cuda.empty_cache()
    del features
    del label

  average_loss = running_loss/len(train_loader.dataset)
  print(f"Epoch:{epoch}, Training Loss:{average_loss}")
  wandb.log({"Training Average Loss":average_loss })

In [8]:
def validate_model(model, val_loader, loss_function, epoch_index, batch_size):
  model.eval()
  running_loss = 0.0

  with torch.no_grad():
    for features, labels in val_loader: 
      features, labels = features.to(device), labels.to(device)
      
      output = model(features)

      loss = loss_function(output, labels)
      running_loss += loss.item()
      
  average_loss = running_loss/len(val_loader.dataset)  
  print(f"Epoch:{epoch_index}, Validation Loss:{average_loss}")
  wandb.log({"Validation Average Loss":average_loss, "Epoch":epoch_index})
  return(average_loss)

In [9]:
  def save_model( model, optimizer, path, scheduler, epoch):
    path = path + str(epoch) + '.pt'
    torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict' : scheduler.state_dict(),
    }, path)

In [10]:
def deploy_model(model, epoch_total, train_loader, optimizer, loss_function, validation_loader, batch_size, scheduler, path):
  for epoch_index in range(epoch_total):
    train_model(model, train_loader, optimizer, loss_function, epoch_index)
    validation_average_loss = validate_model(model,validation_loader,loss_function,epoch_index,batch_size)
    scheduler.step(validation_average_loss)
    #save_model(model, optimizer, path, scheduler, epoch_index)

#Training Model 

In [11]:
config = {
    'epochs' : 9,
    'lr' : .002,
    'optimizer' : 'adam',
    'batch_size' : 16,
    'schedular' : 'ReduceLROnPlateau',
    'weight_decay' : 5e-6,
    'hidden_layer_size_0' : 3*2048,
    'hidden_layer_size_1' : 2*2048,
    'hidden_layer_size_2' : 2*2048,
    'hidden_layer_size_3' : 2*1024,
    'hidden_layer_size_4' : 1*1024,
    'hidden_layer_size_5' : 1*1024,
    'class_size' : 2,
    'dropout': .40,
    'patience' : 3,
    'run_name' : 'run0/',
    'factor' : .4
}

In [12]:
pwd

'/content'

In [17]:
save_path = '/content/gdrive/MyDrive' + config['run_name']
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device is on",device)

nn = ANNModel(config["dropout"], config['hidden_layer_size_0'], config['hidden_layer_size_1'], config['hidden_layer_size_2'],config['hidden_layer_size_3'],config['hidden_layer_size_4'],config['hidden_layer_size_5'],2)
nn = nn.to(device)

train_data_path = '/content/gdrive/MyDrive/bdf_train.csv'
train_eduction_dataset = education_dataset(train_data_path)
train_education_dataloader = create_data_loader(train_eduction_dataset, shuffle = True)

test_data_path = '/content/gdrive/MyDrive/bdf_test.csv'
test_dataset = education_dataset(test_data_path)
test_education_dataloader = create_data_loader(test_dataset, shuffle = False)

optimizer = torch.optim.Adam(nn.parameters(), lr = config['lr'], weight_decay=config['weight_decay'])
loss_function = torch.nn.CrossEntropyLoss() 
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=config['patience'], factor=config['factor'])

Device is on cpu


In [18]:
wandb.init(project="DM Project", config=config)
deploy_model(nn, config['epochs'], train_education_dataloader, optimizer, loss_function, test_education_dataloader, config['batch_size'], scheduler, save_path )
wandb.finish()

Epoch:0, Training Loss:0.018876512041441047
Epoch:0, Validation Loss:0.03857769782726581
Epoch:1, Training Loss:0.009934734172674042
Epoch:1, Validation Loss:0.016226706596521232
Epoch:2, Training Loss:0.006142630340506346
Epoch:2, Validation Loss:0.015759949042246892
Epoch:3, Training Loss:0.00673054029486772
Epoch:3, Validation Loss:0.011175692539948684
Epoch:4, Training Loss:0.005761976186939747
Epoch:4, Validation Loss:0.012150975374075083
Epoch:5, Training Loss:0.005742394281030849
Epoch:5, Validation Loss:0.014128512602586013
Epoch:6, Training Loss:0.005207852017213384
Epoch:6, Validation Loss:0.01173876294722924
Epoch:7, Training Loss:0.004790106547820545
Epoch:7, Validation Loss:0.012711953199826754
Epoch:8, Training Loss:0.0050467652392525205
Epoch:8, Validation Loss:0.012018442612427932


Epoch,▁▂▃▄▅▅▆▇█
Training Average Loss,█▄▂▂▁▁▁▁▁
Validation Average Loss,█▂▂▁▁▂▁▁▁
Epoch,8
Training Average Loss,0.00505
Validation Average Loss,0.01202


#Inference 

In [19]:
def predict_classes(model,test_loader):
  
  with torch.no_grad():
    model.eval()
    true_label = []
    predicted_classes_total = []
    
    for x, y in test_loader:
      x = x.to(device)
      test_out = model(x)
      prediction_logits, prediction_classes = torch.max(test_out,1)
      prediction_classes = prediction_classes.tolist()
      predicted_classes_total.extend(prediction_classes)

      y = y.tolist()
      true_label.extend(y)

  return(predicted_classes_total, true_label)

    
predicted_classes_total, true_label = predict_classes(nn,test_education_dataloader)

In [20]:
total_correct = 0
for index, prediction in enumerate(predicted_classes_total):
  if prediction == true_label[index]:
    total_correct+=1
  else:
    pass
  
print("Total correct=",total_correct )
print("Accuracy =",total_correct/len(predicted_classes_total))

Total correct= 96
Accuracy = 0.7384615384615385
